# Лабораторная работа №4
## Задание:
1 . Провести классификацию найденного датасета, методами наивного Байеса  . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## Решение:
#### Подключение библиотек


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.under_sampling import RandomUnderSampler

#### Загрузка данных и вывод датасета


In [2]:
Dataset = pd.read_csv('property_assessment.csv')
print(Dataset.head())

         PID        CM_ID     GIS_ID  ST_NUM       ST_NAME         CITY  \
0  100018006  100018000.0  100018000   239.0  Lexington ST  EAST BOSTON   
1  100018008  100018000.0  100018000   239.0  Lexington ST  EAST BOSTON   
2  100019006  100019000.0  100019000   241.0  Lexington ST  EAST BOSTON   
3  100024002  100024000.0  100024000   249.0  Lexington ST  EAST BOSTON   
4  100041006  100041000.0  100041000   224.0  PRINCETON ST  EAST BOSTON   

   ZIP_CODE  BLDG_SEQ  NUM_BLDGS  LUC  ... FULL_BTH HLF_BTH KITCHENS TT_RMS  \
0    2128.0         1          1  102  ...      2.0     0.0      1.0    5.0   
1    2128.0         1          1  102  ...      2.0     0.0      1.0    4.0   
2    2128.0         1          1  102  ...      2.0     0.0      1.0    4.0   
3    2128.0         1          1  102  ...      1.0     0.0      1.0    5.0   
4    2128.0         1          1  102  ...      2.0     0.0      1.0    5.0   

     KITCHEN_TYPE           HEAT_TYPE         AC_TYPE FIREPLACES  NUM_PARK

### Создаем искомый столбец

In [3]:
label_encoder = LabelEncoder()
Dataset['COND_ENC'] = label_encoder.fit_transform(Dataset['OVERALL_COND'])

### Нормализация данных

In [4]:
Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']] = Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']].applymap(lambda x: x.replace(',', '') if isinstance(x, str) else x).apply(pd.to_numeric, errors='coerce')
numeric_columns = Dataset.select_dtypes(include='number').columns.tolist()
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10321 entries, 0 to 10320
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PID          10321 non-null  int64  
 1   CM_ID        10321 non-null  float64
 2   GIS_ID       10321 non-null  int64  
 3   ST_NUM       10321 non-null  float64
 4   ZIP_CODE     10321 non-null  float64
 5   BLDG_SEQ     10321 non-null  int64  
 6   NUM_BLDGS    10321 non-null  int64  
 7   LUC          10321 non-null  int64  
 8   RES_FLOOR    10321 non-null  float64
 9   GROSS_AREA   10321 non-null  float64
 10  LIVING_AREA  10321 non-null  float64
 11  LAND_VALUE   10321 non-null  int64  
 12  BLDG_VALUE   10321 non-null  int64  
 13  SFYI_VALUE   10321 non-null  int64  
 14  TOTAL_VALUE  10321 non-null  int64  
 15  YR_BUILT     10321 non-null  float64
 16  YR_REMODEL   10321 non-null  float64
 17  BED_RMS      10321 non-null  float64
 18  FULL_BTH     10321 non-null  float64
 19  HLF_

### Поиск лучших метрик

In [5]:
X = Dataset.drop('COND_ENC', axis=1)
y = Dataset['COND_ENC']

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7]}

# Инициализация модели наивного Байеса
nb = GaussianNB()

# Инициализация Grid Search
grid_search = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Поиск лучших параметров на обучающем наборе данных
grid_search.fit(X_train, y_train)

# Обучение модели на обучающем наборе данных
nb.fit(X_train, y_train)

# Предсказание на тестовых данных
predictions = nb.predict(X_test)

# Получение лучших параметров и лучшей модели
best_params = grid_search.best_params_

best_model = grid_search.best_estimator_

# Оценка качества модели на тестовом наборе
accuracy = accuracy_score(y_test, predictions)
print(f'Точность: {accuracy}')

# Вывод отчета о классификации
print(classification_report(y_test, predictions))

print("Лучшие параметры:", best_params)


Точность: 0.7108958837772397
              precision    recall  f1-score   support

           0       0.76      0.95      0.84      1526
           1       0.20      0.19      0.19        27
           2       0.00      0.00      0.00        28
           3       1.00      1.00      1.00         1
           4       0.21      0.02      0.04       462
           5       0.10      0.38      0.16        21

    accuracy                           0.71      2065
   macro avg       0.38      0.42      0.37      2065
weighted avg       0.61      0.71      0.63      2065

Лучшие параметры: {'var_smoothing': 1e-09}
